In [1]:
import torch

In [2]:
def _get_subsample_centroid(Xtr: torch.Tensor, ytr: torch.Tensor, subsample_size: int) -> tuple[torch.Tensor, torch.Tensor]:
    # randomly sample a point from the training set
    idx = torch.randint(0, Xtr.shape[0], (1,))
    centroid = Xtr[idx]

    print(f"idx = {idx.item()}")
    print(f"centroid = {centroid}")

    # find the subset of points closest to the centroid
    distances = torch.cdist(Xtr, centroid.view(1, -1)).squeeze(-1)
    print(f"distances = {distances}")
    _, indices = torch.topk(distances, k=subsample_size,
                            largest=False, sorted=False)
    print(f"indices = {indices}")
    return Xtr[indices], ytr[indices]

In [3]:
n = 10
d = 2

Xtr = torch.randn(n, d)
ytr = torch.randn(n)
subsample_size = 5
Xtr_subsample, ytr_subsample = _get_subsample_centroid(Xtr, ytr, subsample_size)
print("Xtr_subsample:", Xtr_subsample)
print("ytr_subsample:", ytr_subsample)

idx = 8
centroid = tensor([[-0.0853,  0.7085]])
distances = tensor([0.6074, 0.3207, 0.7458, 2.4428, 1.2951, 0.5726, 0.3329, 1.6718, 0.0000,
        1.6428])
indices = tensor([5, 1, 8, 6, 0])
Xtr_subsample: tensor([[ 0.4054,  0.4133],
        [-0.0410,  1.0261],
        [-0.0853,  0.7085],
        [-0.0824,  0.3756],
        [-0.3851,  1.2367]])
ytr_subsample: tensor([-1.0992,  0.2130, -0.3095, -0.1827,  0.5532])


In [6]:
Xtr

tensor([[-0.3851,  1.2367],
        [-0.0410,  1.0261],
        [-0.3718,  1.3970],
        [-0.3423, -1.7208],
        [ 0.3583, -0.5082],
        [ 0.4054,  0.4133],
        [-0.0824,  0.3756],
        [-1.3519, -0.3825],
        [-0.0853,  0.7085],
        [-0.7558, -0.7913]])

In [7]:
ytr

tensor([ 0.5532,  0.2130, -0.4564,  0.7508,  0.5927, -1.0992, -0.1827, -0.5880,
        -0.3095,  0.3414])

In [4]:
Xtr[8]

tensor([-0.0853,  0.7085])

In [5]:
torch.cdist(Xtr, Xtr[8].view(1, -1)).squeeze(-1)

tensor([0.6074, 0.3207, 0.7458, 2.4428, 1.2951, 0.5726, 0.3329, 1.6718, 0.0000,
        1.6428])